# Bank Ambiguous Name Pairing

In [1]:
pip install fuzzywuzzy

Note: you may need to restart the kernel to use updated packages.


In [2]:
import re
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz

- https://www.datacamp.com/community/tutorials/fuzzy-string-python


* Import your Dataset

In [3]:
amb_data = pd.read_csv('your dataset')
# you are going to compare company A and B in the following codes
# name your company A column as "namefull", and B column as "company". So that you can use all the function I defined below

NameError: name 'pd' is not defined

## Helper Functions

In [31]:
def listToString(s):  
    
    # initialize an empty string 
    str1 = ""  
    
    # traverse in the string   
    for ele in s:  
        str1 += ele + ' '
    
    # return string   
    return str1  

In [ ]:
def info(df):
    info = []
    full = df['namefull'].tolist()
    cname = df['company'].tolist()
    for i in np.arange(len(full)):
        pair = [full[i], cname[i]]
        info.append(compare_message(pair[0], pair[1]))
    return info

In [ ]:
def dummy_generator(df):
    info = df['info'].tolist()
    output = []
    for i in info:
        if "Matched" in i:
            output.append(1)
        else:
            output.append(0)
    return output

In [ ]:
def group_assgin(df):
    info = df['info'].tolist()
    output = []
    for i in info:
        # Group 1: 100% Correct
        if 'Matched' in i:
            output.append('Group 1')
        elif 'Must Conflict' in i:
            output.append('Group 1')
        # Group 2: Manual Check needed
        else:
            output.append('Group 2')
    return output

# Filter Function

In [1]:
# Conf: a nested list that contains pair of strings that must be conflict with each other
conf = [['WESTERN', 'EASTERN'], ['NORTHEN', 'SOUTHEN'], 
        ['NORTH', 'SOUTH'], ['EAST', 'WEST'], 
        ['SOUTHEASTERN', 'EASTERN'], ['TEJAS', 'TEXAS'], 
        ['TEXAN', 'TEJAS'], ['BELLVILLE', 'BEEVILLE']]


# Inter: a nested list that contains pair of strings that are interchangable when matching Banks names
inter = [['NEW YORK', 'NY'], ['1ST', 'FIRST'], 
         ['F.S.B.', 'FSB'], ['IOWA', 'IA'], 
         ['CCNB –', ''], ['/ BNY', ''], 
         ['SAVINGS', 'SAVING'], ['SAVING', 'SAVIN'], 
         ['SAVINGS', 'SAVINS'], ['PROAMERICA', 'PROAMÉRICA'], 
         ['BANK', 'BANKS'], ['CO.', 'COMPANY '],
        ['ESPIRITO', 'ESPÍRITO'], ['&', 'AND'], 
        ['CENTER', 'CENTRE'], ['GRAVETTE', 'GRAVETT']]

In [2]:
# Compare_message: compare string A and B. Return a message indicating if A and B is a correct match or not with specific reasons
# A: Is the String A

def compare_message(A, B, conflicts = conf, interchange = inter):
    # Interchangable fix: add interchangable words
    for i in interchange:
        A = A.upper()
        B = B.upper()
        i1 = i[0]
        i2 = i[1]
        case_1 = i1 in A and i2 in B
        case_2 = i1 in B and i2 in A
        if any([case_1, case_2]):
            start = '^(' + i1 + ')\s'
            mid = '\s+(' + i1 + ')$'
            end = '\s+(' + i1 + ')\,*\s+'
            reg2 = " " + i2 + ' '
            A = re.sub(start, reg2, A)
            A = re.sub(mid, reg2, A)
            A = re.sub(end, reg2, A)
            B = re.sub(start, reg2, B)
            B = re.sub(mid, reg2, B)
            B = re.sub(end, reg2, B)

    # 100% Matched
    a, b = A, B
    a, b = a.strip(), b.strip()
    if a==b:
        return "Matched"
    
    # MUST CONFLICT CASES
    a, b = A.upper(), B.upper()
    for i in conflicts:
        j1 = i[0]
        j2 = i[1]
        if j1 in a and j2 in b:
            return "(" + j1 + ')' +' vs '+'(' + j2 + ') ' + 'Must Conflict'
        if (j1 in b) and (j2 in a):
            return "(" + j1 + ')' +' vs '+'(' + j2 + ') ' + 'Must Conflict'
    
    # Case of Co. Co and Company
    a, b = A.upper(), B.upper()
    a, b = a.strip(), b.strip()
    a, b = re.sub('CO\.$', ' ', a), re.sub('CO\.$', ' ', b)
    a, b = re.sub('CO$', ' ', a), re.sub('CO$', ' ', b)
    a, b = re.sub('COMPANY$', ' ', a), re.sub('COMPANY$', ' ', b)
    a, b = re.sub('INC\.*$', ' ', a), re.sub('INC\.*$', ' ', b)
    a, b = a.replace('.', ''), b.replace('.', '')
    a, b = a.replace(',', ''), b.replace(',', '')
    a, b = a.replace('-', ' '), b.replace('-', ' ')
    a, b = re.sub('\s+','',a), re.sub('\s+','',b)
    if a == b:
        return "Company, Co., Co, INC., Matched"
    
    # Case of FSB
    a, b = A.upper(), B.upper()
    a, b = a.strip(), b.strip()
    a, b = re.sub('FSB$', ' ', a), re.sub('FSB$', ' ', b)
    a, b = a.replace('.', ''), b.replace('.', '')
    a, b = a.replace(',', ''), b.replace(',', '')
    a, b = a.replace('-', ' '), b.replace('-', ' ')
    a, b = re.sub('\s+','',a), re.sub('\s+','',b)
    if a == b:
        return "W and W/O FSB, Matched"
    
    # Case of SSB
    a, b = A.upper(), B.upper()
    a, b = a.strip(), b.strip()
    a, b = re.sub('SSB$', ' ', a), re.sub('SSB$', ' ', b)
    a, b = a.replace('.', ''), b.replace('.', '')
    a, b = a.replace(',', ''), b.replace(',', '')
    a, b = a.replace('-', ' '), b.replace('-', ' ')
    a, b = re.sub('\s+','',a), re.sub('\s+','',b)
    if a == b:
        return "W and W/O SSB, Matched"

    # Matched Cases
    a, b = A.replace('.', ''), B.replace('.', '')
    a, b = a.replace(',', ''), b.replace(',', '')
    a, b = a.replace('-', ' '), b.replace('-', ' ')
    a, b = a.strip(), b.strip()
    a, b = re.sub('\s+','',a), re.sub('\s+','',b)
    if fuzz.ratio(A.lower(), B.lower()) >= 99 or a.upper() == b.upper():
        return "Non-alphanumeric, Matched"
    
    # Double Space Case
    a, b = re.sub('\s+',' ',A), re.sub('\s+',' ',B)
    if a.upper()==b.upper():
        return 'Double space, Matched'
    
    # Case of And and &
    if '&' in A or '&' in B:
        if '&' in A:
            a = A.replace('&', ' and ')
        if '&' in B:
            b = B.replace('&', ' and ')
        a, b = re.sub('\s','',a), re.sub('\s','',b)
        a, b = re.sub('\W','',a), re.sub('\W','',b)                         
        if a.upper() == b.upper():
            return "(&) vs (AND), Matched"
        
    # Case 'The'
    a, b = A.upper(), B.upper()
    if 'THE' in a or 'THE' in a:
        if 'THE' in a:
            a = a.replace('THE', '')
        if 'THE' in b:
            b = b.replace('THE', '')
        a, b = a.replace('-', ' '), b.replace('-', ' ')
        a, b = a.replace('OF', ' '), b.replace('OF', ' ')
        a, b = re.sub('\s+','',a), re.sub('\s+','',b)
        if a.upper() == b.upper():
            return "W and W/O (THE), Matched"
    
    # Single Space Case
    a, b = re.sub('\s','',A), re.sub('\s','',B)
    if a.upper()==b.upper():
        return 'Single space, Matched'
    
    # IS IT ABOUT SOME NON-ALPHANUMERIC CHARACTERS?
    a, b = A, B
    a, b = re.sub('\W','',a), re.sub('\W','',b)
    if a.upper()==b.upper():
        return 'Non-alphanumeric, Matched'
    
    # Case of '-', 'in', 'of'
    a, b = A.upper(), B.upper()
    a, b = a.replace('-', ''), b.replace('-', '')
    a, b = a.replace('，', ''), b.replace('，', '')
    a, b = a.replace('THE', ''), b.replace('THE', '')
    a, b = a.replace('IN', ''), b.replace('IN', '')
    a, b = a.replace('OF', ''), b.replace('OF', '')
    a, b = re.sub('\W','',a), re.sub('\W','',b)
    a, b = re.sub('\s+','',a), re.sub('\s+','',b)
    if a.upper()==b.upper():
        return 'Non-alphanumeric, Matched'
    
    # Case of N.A. 
    a, b = A.upper(), B.upper()
    if 'NA' in a or 'NA' in b:
        a, b = a.strip(), b.strip()
        a, b = re.sub('\s+NA$','',a), re.sub('\s+NA$','',b)
        a, b = a.replace(',', ''), b.replace(',', '')
        a, b = a.replace('-', ''), b.replace('-', '')
        if a == b:
            return 'W and W/O (N.A.), Matched'
    a, b = re.sub('\s+','',a), re.sub('\s+','',b)
    if 'N.A.' in a or 'N.A.' in b or 'N.A' in a or 'N.A' in b:
        a, b = a.replace('N.A.', ''), b.replace('N.A.', '')
        a, b = a.replace('N.A', ''), b.replace('N.A', '')
        a, b = a.replace(',', ''), b.replace(',', '')
        a, b = a.replace('-', ''), b.replace('-', '')
        if a == b:
            return 'W and W/O (N.A.), Matched' 
    
    # Case of S.A.
    a, b = A.upper(), B.upper()
    a, b = re.sub('\s+','',a), re.sub('\s+','',b)
    if 'S.A.' in a or 'S.A.' in b or 'S.A' in a or 'S.A' in b:
        a, b = a.replace('S.A.', ''), b.replace('S.A.', '')
        a, b = a.replace('S.A', ''), b.replace('S.A', '')
        a, b = a.replace(',', ''), b.replace(',', '')
        a, b = a.replace('-', ''), b.replace('-', '')
        if a == b:
            return 'W and W/O (S.A.), Matched'
    
    # IS IT ABOUT SOME WORDS BEING DIFFERENT?
    a, b = A, B
    # Added: Case where period and comma messed up the matching
    a, b = A.replace('.', ' '), B.replace('.', ' ')
    a, b = a.replace(',', ' '), b.replace(',', ' ')
    a, b = a.replace('-', ''), b.replace('-', '')
    a, b = a.replace('N.A.', ''), b.replace('N.A.', '')
    a, b = a.replace('N.A', ''), b.replace('N.A', '')
    a, b = a.strip(), b.strip()
    a, b = re.sub('\s+',' ',a), re.sub('\s+',' ',b)
    a, b = a.upper(), b.upper()
    a_words, b_words = a.split(' '), b.split(' ')
    
    # SAME NUMBER OF WORDS ?
    if len(a_words)==len(b_words):
        a_specific_words, b_specific_words = [], []
        for i in range(len(a_words)):
            if a_words[i]!=b_words[i]:
                a_specific_words.append(a_words[i])
                b_specific_words.append(b_words[i])
        a_msg = '; '.join(a_specific_words)
        b_msg = '; '.join(b_specific_words)
        if 'FIRST' in a_specific_words or 'FIRST' in b_specific_words:
            return '(%s) vs (%s), Must Conflict: First vs Other words' % (a_msg,b_msg)
        return '(%s) vs (%s)' % (a_msg,b_msg)
    
    # ONE AND ONLY ONE EXTRA DISTINCT WORD
    if abs(len(a_words)-len(b_words))==1:
        # Case of THE
        if "THE" in a or "THE" in b:
            if "THE" in a_words:
                a_words.remove('THE')
            if "THE" in b_words:
                b_words.remove('THE')
            if a_words==b_words:
                return "W and W/O (THE), Matched"

        # Case of 'The First National Bank Of'
        contain = 'FIRST NATIONAL BANK'
        if contain in a and contain in b:
            a, b = a.replace('THE', ''), b.replace('THE', '')
            a, b = a.replace('OF', ''), b.replace('OF', '')
            a, b = a.replace('IN', ''), b.replace('IN', '')
            a, b = a.replace(contain, ''), b.replace(contain, '')
            a, b = a.strip(), b.strip()
            return '(' + a + ')' + ' vs ' + '(' + b + ')'  
            
        a_specific_words = list(filter(lambda x: x not in b_words, a_words))
        b_specific_words = list(filter(lambda x: x not in a_words, b_words))
        
        # Other Difference
        in_a = listToString(a_specific_words).strip().replace(' ', '; ')
        in_b = listToString(b_specific_words).strip().replace(' ', '; ')
        if 'FIRST' in in_a or 'FIRST' in in_b:
            return '(' + in_a + ') vs (' + in_b + ')' + ', Must Conflict: First vs Other words'
        return '(' + in_a + ') vs (' + in_b + ')'
        
        if len(a_specific_words)+len(b_specific_words)==1:
            if len(a_words)>len(b_words):
                shorter_seq = b_words
                longer_seq  = a_words
            elif len(a_words)<len(b_words):
                shorter_seq = a_words
                longer_seq  = b_words

            # corner difference
            if longer_seq[0]!=shorter_seq[0]:
                return '{%s} %s' %(longer_seq[0],' '.join(longer_seq[1:]))
            if longer_seq[-1]!=shorter_seq[-1]:
                return '%s {%s}' %(' '.join(longer_seq[:-1]),longer_seq[-1])
         
            # interior difference
            N = len(longer_seq)
            for i in range(1,N-1):
                if longer_seq[i]!=shorter_seq[i]:
                    return '%s {%s} %s' %(' '.join(longer_seq[:i]),longer_seq[i],' '.join(longer_seq[i+1:]))
                
    if abs(len(a_words)-len(b_words)) != 1:
        a_specific_words = list(filter(lambda x: x not in b_words, a_words))
        b_specific_words = list(filter(lambda x: x not in a_words, b_words))
        in_a = listToString(a_specific_words).strip().replace(' ', '; ')
        in_b = listToString(b_specific_words).strip().replace(' ', '; ')
        return '(' + in_a + ') vs (' + in_b + ')' + 'Two or More Diffs'
    return 'Unidentified'

In [74]:
amb_data['info'] = info(amb_data)

In [76]:
amb_data['correct_match'] = dummy_generator(amb_data)

In [78]:
amb_data['group'] = group_assgin(amb_data)

In [83]:
amb_data = amb_data.drop([0], 0)

# Expore completed DS

In [85]:
amb_data.to_csv('filtered_data.csv')